In [1]:
import numpy as np 
import tensorflow as tf

In [2]:
import tensorflow_datasets as tfds

In [4]:
mnist_dataset, mnist_info = tfds.load(name='mnist',with_info=True, as_supervised=True)

In [5]:
mnist_dataset[1]

KeyError: 1

In [12]:
print(mnist_dataset)
mnist_info.splits['train'].num_examples

{'test': <PrefetchDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>, 'train': <PrefetchDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>}


60000

In [15]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples,tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples,tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

scaled_test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = scaled_test_data.batch(num_test_samples)

vaidation_input, validation_target = next(iter(validation_data))

In [16]:
28*28

784

In [17]:
input_size = 784
output_size = 10
hidden_layer_size = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape = (28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation= 'relu'),
                            tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                            tf.keras.layers.Dense(output_size,activation='softmax')
                            ])

In [18]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics= ['accuracy'])

In [19]:
NUM_EPOCH =5

model.fit(train_data, epochs=NUM_EPOCH, validation_data=(vaidation_input, validation_target), verbose = 2)

Epoch 1/5
540/540 - 9s - loss: 0.4039 - accuracy: 0.8858 - val_loss: 0.2131 - val_accuracy: 0.9398
Epoch 2/5
540/540 - 6s - loss: 0.1816 - accuracy: 0.9459 - val_loss: 0.1614 - val_accuracy: 0.9527
Epoch 3/5
540/540 - 7s - loss: 0.1390 - accuracy: 0.9586 - val_loss: 0.1315 - val_accuracy: 0.9625
Epoch 4/5
540/540 - 6s - loss: 0.1113 - accuracy: 0.9659 - val_loss: 0.1178 - val_accuracy: 0.9650
Epoch 5/5
540/540 - 6s - loss: 0.0938 - accuracy: 0.9712 - val_loss: 0.1005 - val_accuracy: 0.9703


In [20]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 20ms/step - loss: 0.1058 - accuracy: 0.9658


In [23]:
print('test_loss = {}, test_accuracy = {}%'.format(test_loss,test_accuracy*100))

test_loss = 0.10577286034822464, test_accuracy = 96.57999873161316%
